In [ ]:
"""--------------------------------TASK3--------------------------------"""
import pickle
import json

def write_in_pickle(data, path):
    """
    Write data into a file formed by pickle.
    """
    output = open(path, 'wb')
    pickle.dump(data, output)
    output.close()

def load_pickle(path):
    """
    Load data from a file formed by pickle.
    Return the data in the form of original.
    """
    pkl_file = open(path, 'rb')
    data = pickle.load(pkl_file)
    pkl_file.close()
    return data

def load_json(dir):
    """
    Load the json data and return. 
    """
    data = []
    with open(dir, 'r') as f:
        while True:
            a = f.readline()
            if not a:
                break
            b = json.loads(a)
            data.append(b)
    return data

# Get all papers from allpaper.txt
papers = load_json('/Users/zhengwei/Desktop/allpaper.txt')

In [ ]:
"""
Generate the cooperations directionary and save in file in form of pickle.
{key: author name, value: a list [c1_name, c2_name, ...]}
"""
cooperation_authors = {}
for paper in papers:
    if paper["year"] > 2012:
        continue
    for author in paper["authors"]:
        if author not in cooperation_authors:
            cooperation_authors[author] = []
        for author_coor in paper["authors"]:
            if author_coor != author and author_coor not in cooperation_authors[author]:
                cooperation_authors[author].append(author_coor)
write_in_pickle(cooperation_authors, "/Users/zhengwei/Desktop/coor_author.pkl")

In [ ]:
"""
Generate two dictionnary big_dict and end_dic.
big_dict: {key: the name of conference, value: [id1, id2, ...]}
end_dic: {key: the name of author, value: {key: year, value: a dim-length list, dim is the
          number of conferences and each item takes 0 or 1, 1: appear, 0: nope}}
"""
import os
import copy

file_dir = ("/Users/zhengwei/Desktop/all/社交网络挖掘/social_network/papers/")
file_list = [files for root, dirs, files in os.walk(file_dir)][0]
big_dict = {}
end_dic = {}
for item in file_list:
    temp_file = load_json(file_dir + item)
    domain = item[:item.index('.')]

    big_dict[domain] = []
    for entry in temp_file:
        big_dict[domain].append(entry["id"])

domain_list = list(big_dict.keys())
year = [i for i in range(1965, 2018)]
init_vec = [[0]*len(domain_list) for i in range(len(year))]
init_dic = dict(zip(year, init_vec))
for paper in papers:
    domain_index = -1
    paper_id = paper["id"]
    for i in range(len(domain_list)):
        if paper_id in big_dict[domain_list[i]]:
            domain_index = i
            break
    for author in paper["authors"]:
        if author not in end_dic:
            end_dic[author] = copy.deepcopy(init_dic)
        end_dic[author][paper["year"]][domain_index] += 1

In [ ]:
"""
Train the transfer matrix of order dimension, order stands for previous years.
Part the data into train dataset and test dataset, taking the 2012 as the boundary.
"""
import numpy as np
order = 10
tran_matrix = {}
dim = len(domain_list)
for author in end_dic:
    tran_matrix[author] = [[[0,0,0,0] for j in range(dim)]for i in range(dim)]  # (1->1, 1->0, 0->1, 0->0)
    first_paper = 1
    old_year_data = []
    for year, new_year_data in end_dic[author].items():
        if year > 2012:
            break
        if first_paper:
            if sum(new_year_data):
                continue
            else:
                first_paper = 0
                year_grep = 1
                old_year_data.append(new_year_data)
                continue
        if len(old_year_data) < 10:
            old_year_data.append(new_year_data)
            if year < 2012:
                continue
        five_year_data = np.array(old_year_data).sum(axis=0)
        for i in range(len(five_year_data)):
            data1 = five_year_data[i]
            for j in range(len(new_year_data)):
                data2 = new_year_data[j]
                if data1:
                    if data2:
                        tran_matrix[author][i][j][0] += 1
                    else:
                        tran_matrix[author][i][j][1] += 1
                else:
                    if data2:
                        tran_matrix[author][i][j][2] += 1
                    else:
                        tran_matrix[author][i][j][3] += 1
        old_year_data.append(new_year_data)
        old_year_data = old_year_data[1:]
    for i in range(dim):
        for j in range(dim):
            temp = tran_matrix[author][i][j]
            tran_matrix[author][i][j] = [temp[0]/(temp[0] + temp[1] + 1e-10), temp[2]/(temp[2] + temp[3] + 1e-10)]
write_in_pickle(tran_matrix, "/Users/zhengwei/Desktop/transfer_matrix_10.pkl")

In [ ]:
"""
Employ the MDP model into the train dataset to compare the test dataset and the predicted result.
Calculate the accuracy, precision, recall and F1-score for two issues: 
one is whether an author will appear a paper next year, the other is whether the paper will hit the conference. 
"""
import numpy as np
dim = 21
end_result = {}
for year in range(2013, 2018):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    TP1 = 0
    FP1 = 0
    TN1 = 0
    FN1 = 0
    for author in end_dic:
        test_vec = end_dic[author][year]
        train_vec = list(np.array([end_dic[author][i] for i in range(year-10,year)]).sum(axis=0))
        predict_vec = [0] * dim
        for col in range(dim):
            temp_vec = np.array(tran_matrix[author])[:,col]
            for i in range(dim):
                if i == col:
                    predict_vec[col] += (temp_vec[i][0] if train_vec[i] else temp_vec[i][1]) * 0.9 * 3
                else:
                    predict_vec[col] += (temp_vec[i][0] if train_vec[i] else temp_vec[i][1]) * 0.005 * 3
        predict_vec = [0 + int(item > 0.5) for item in predict_vec]
        expected = sum(test_vec)
        predicted = sum(predict_vec)
        if expected:
            if predicted:
                TP += 1
            else:
                FN += 1
        else:
            if predicted:
                FP += 1
            else:
                TN += 1

        for i in range(dim):
            expected_v = test_vec[i]
            predicted_v = predict_vec[i]
            if expected_v:
                if predicted_v:
                    TP1 += 1
                else:
                    FN1 += 1
            else:
                if predicted_v:
                    FP1 += 1
                else:
                    TN1 += 1
    result = [[0,0,0,0],[0,0,0,0]]
    result[0][0] = (TP + FN)/(TP + TN + FP + FN)
    result[0][1] = TP/(TP + FP)
    result[0][2] = TP/(TP + FN)
    result[0][3] = 2/(1/result[0][1] + 1/result[0][2])
    result[1][0] = (TP1 + TN1)/(TP1 + TN1 + FP1 + FN1)
    result[1][1] = TP1/(TP1 + FP1)
    result[1][2] = TP1/(TP1 + FN1)
    result[1][3] = 2/(1/result[1][1] + 1/result[1][2])
    end_result[year] = result
write_in_pickle(end_result, "/Users/zhengwei/Desktop/end_result_10.pkl")